In [7]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

In [8]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-05-15 05:14:03--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.30MB/s    in 0.2s    

2021-05-15 05:14:03 (5.30 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL_AWS_Home").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [10]:
# Configuration for RDS instance
mode="overwrite"
jdbc_url = "jdbc:postgresql://big-data-challenge.csldkrhc63yg.ap-southeast-2.rds.amazonaws.com:5432/big_data_db"
config = {"user":"postgres",
          "password": "password",
          "driver":"org.postgresql.Driver"}

In [11]:
from pyspark import SparkFiles
#load software reviews data from s3.amazonaws.com
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

home_df = spark.read.option("header","true").csv(SparkFiles.get("amazon_reviews_us_Home_v1_00.tsv.gz"), inferSchema=True, dateFormat="yyyy-MM-dd",sep="\t")
home_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

# **Drop duplicates and incomplete rows**

In [12]:
print(home_df.count())
home_df = home_df.dropna()
print(home_df.count())
home_df = home_df.dropDuplicates()
print(home_df.count())

6221559
6220615
6220615


# **Examine the schema**

In [13]:
home_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



# **Transfor Dataframe type**

In [14]:
import datetime
home_df = home_df.withColumn("review_date",home_df["review_date"].cast("date"))
home_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24502943|R100G2BP1C58X8|B002JM0DDG|     881311856|Oregon Scientific...|            Home|          2|            0|          0|   N|                Y| not a good choice..|this is my second...| 2014-09-08|
|         US|   40359545|R100RCND4M6WL8|B003XDTWN2|     996572389|Lasko My Heat Per...|            Home|          5|    

In [15]:
home_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



# **Create Dataframe for review_id_table**

In [16]:
review_id_df_new = home_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df_new.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100G2BP1C58X8|   24502943|B002JM0DDG|     881311856| 2014-09-08|
|R100RCND4M6WL8|   40359545|B003XDTWN2|     996572389| 2012-11-13|
| R101F576XH7WV|   39854168|B0094KU62Y|     791679766| 2014-12-07|
|R101PD18H5UK5C|   23135804|B000ASB3RG|     392799790| 2008-01-18|
|R101Z27PX08DSF|   20118795|B002HGEPEM|      30868741| 2014-11-21|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [17]:
review_id_df_pre = spark.read.jdbc(url=jdbc_url,table="review_id_table", properties=config)
review_id_df = review_id_df_new.union(review_id_df_pre)

In [18]:
print(review_id_df.count())
review_id_df = review_id_df.dropDuplicates(["review_id"])
print(review_id_df.count())

6220615
6220615


# **Create Dataframe for products_table**

In [19]:
products_df_new = home_df.select(["product_id", "product_title"])
products_df_new.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B002JM0DDG|Oregon Scientific...|
|B003XDTWN2|Lasko My Heat Per...|
|B0094KU62Y|Miles Kimball Sel...|
|B000ASB3RG|Bissell ProHeat 2...|
|B002HGEPEM|Realtree Unisex A...|
+----------+--------------------+
only showing top 5 rows



In [20]:
products_df_pre = spark.read.jdbc(url=jdbc_url,table="products", properties=config)
products_df=products_df_new.union(products_df_pre)

In [21]:
print(products_df.count())
products_df = products_df.dropDuplicates(["product_id"])
print(products_df.count())

6220615
1074731


# **Create Dataframe for customers_table**

In [22]:
customers_df_new = home_df.select(["customer_id"])
customers_df_new = customers_df_new.groupBy("customer_id").count()
customers_df_new.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   45233105|    1|
|   52773179|   17|
|   46695659|    2|
|   28031940|    7|
|    9582158|    1|
+-----------+-----+
only showing top 5 rows



In [23]:
customers_df_pre = spark.read.jdbc(url=jdbc_url,table="customers", properties=config)
if customers_df_pre.count():
  customers_df=customers_df_new.join(customers_df_pre,on=["customer_id"],how="outer")
  customers_df=customers_df.fillna(0)
  customers_df=customers_df.withColumn('total', sum(customers_df[col] for col in ["count", "customer_count"]))
  customers_df = customers_df.drop("count", "customer_count")
  customers_df = customers_df.withColumnRenamed("total", "customer_count")
  customers_df.show(5)
else:
  customers_df = customers_df_new.withColumnRenamed("count", "customer_count")
  customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45233105|             1|
|   52773179|            17|
|   46695659|             2|
|   28031940|             7|
|    9582158|             1|
+-----------+--------------+
only showing top 5 rows



# **Create Dataframe for vine_table**

In [24]:
vine_df_new = home_df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df_new.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100G2BP1C58X8|          2|            0|          0|   N|
|R100RCND4M6WL8|          5|            1|          1|   N|
| R101F576XH7WV|          5|            0|          0|   N|
|R101PD18H5UK5C|          1|            0|          1|   N|
|R101Z27PX08DSF|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [25]:
vine_df_pre = spark.read.jdbc(url=jdbc_url,table="vine_table", properties=config)
vine_df = vine_df_new.union(vine_df_pre)

In [26]:
print(vine_df.count())
vine_df = vine_df.dropDuplicates(["review_id"])
print(vine_df.count())

6220615
6220615


# **Write DataFrame to RDS**

In [27]:
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [28]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [29]:
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [30]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)